In [1]:
import snap
import numpy as np
from numpy.linalg import inv
from numpy.linalg import norm 

## Simple pagerank

In [2]:
def simple_pagerank(G, eps = 1e-8):
    n = G.GetNodes()
    A = np.zeros((n, n))

    for NI in G.Nodes():
        u = NI.GetId()
        Nu = len(list(NI.GetOutEdges()))
        for v in NI.GetOutEdges():
            A[v, u] = 1 / Nu

    v = np.full((n, 1), 1 / n)
    while True:
        prev = v
        v = A @ v

        if norm(prev - v, ord = 1) < eps:
            return v, A

Returned vector v is stationary distribution of matrix A (sum of its components equals to eiegenvalue = 1) and it describes probabilites of moving to next state (page).

In [3]:
### Example tested by hand
G1 = snap.TNGraph.New()
G1.AddNode(0)
G1.AddNode(1)
G1.AddNode(2)
G1.AddNode(3)
G1.AddEdge(0,1)
G1.AddEdge(0,2)
G1.AddEdge(1,3)
G1.AddEdge(2,0)
G1.AddEdge(2,1)
G1.AddEdge(2,3)
G1.AddEdge(3,2)

v, A = simple_pagerank(G1)
print(v)

[[0.125 ]
 [0.1875]
 [0.375 ]
 [0.3125]]


## Pagerank

In [4]:
def pagerank(G, d = .9, e = None, eps = 1e-8):
    n = G.GetNodes()
    A = np.zeros((n, n))

    for NI in G.Nodes():
        u = NI.GetId()
        Nu = len(list(NI.GetOutEdges()))
        for v in NI.GetOutEdges():
            A[v, u] = 1 / Nu

    if e is None:
        e = 1 / n * np.ones((n, 1))

    B = (d * A) + ((1 - d) * e @ np.ones((1, n)))
    v = np.full((n, 1), 1 / n)
    while True:
        prev = v
        v = B @ v
        d = norm(prev, ord = 1) - norm(v, ord = 1)
        v = v + d * e 
        delta = norm(v - prev, ord = 1)

        if delta < eps:
            return v, B

If we set d to 1.0 we get exactly the same result simple page_rank becasue we don't allow any "teleportation". By default e (E) is uniform distribution over all states (pages). If we choose to use different e we might get "personalized" results.

In [5]:
v, B = pagerank(G1, d = 1.0)

print(v)

[[0.125 ]
 [0.1875]
 [0.375 ]
 [0.3125]]


## Examples

- full graph with 15 vertices. As expected we get equal probabilites for each node.

In [6]:
G2 = snap.GenFull(snap.PNGraph, 15)
v, A = simple_pagerank(G2)
v

array([[0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667],
       [0.06666667]])

- tree with 3 levels where each parent has 2 children,  

In [7]:
G3 = snap.GenTree(snap.PNGraph, 3, 2)
v, A = pagerank(G3)
v

array([[0.35348987],
       [0.11900933],
       [0.11900933],
       [0.11900933],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468],
       [0.03216468]])

- Star graph - center node (first one in vector below) contains edges that point to all 
 other nodes but there is no such node that points back at him. So we can access him only via random "teleportation"
 - trying various d settings.

In [8]:
G4 = snap.GenStar(snap.PNGraph, 15, True)
v, A = pagerank(G4)
v

array([[0.06289308],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621],
       [0.06693621]])

In [9]:
G5 = snap.GenRndPowerLaw(25, 3)
G5 = snap.ConvertGraph(snap.PNGraph, G5)
v, A = pagerank(G5, d = .9)
v

array([[0.02428869],
       [0.02523998],
       [0.04686801],
       [0.02335972],
       [0.0286164 ],
       [0.04149378],
       [0.02428869],
       [0.04445033],
       [0.05437115],
       [0.08560901],
       [0.04149378],
       [0.02341141],
       [0.00414938],
       [0.0286164 ],
       [0.04149378],
       [0.04149378],
       [0.04262175],
       [0.02341141],
       [0.0316663 ],
       [0.0316663 ],
       [0.02335972],
       [0.06114872],
       [0.05437115],
       [0.08537931],
       [0.06713105]])

In [10]:
v, A = pagerank(G5,d = .85)
v

array([[0.02507658],
       [0.02629218],
       [0.04724941],
       [0.02389518],
       [0.02905815],
       [0.04140787],
       [0.02507658],
       [0.04397618],
       [0.05375758],
       [0.08349363],
       [0.04140787],
       [0.02395358],
       [0.00621118],
       [0.02905815],
       [0.04140787],
       [0.04140787],
       [0.04163758],
       [0.02395358],
       [0.03189525],
       [0.03189525],
       [0.02389518],
       [0.0604331 ],
       [0.05375758],
       [0.08321884],
       [0.06658376]])

In [11]:
v, A = pagerank(G5,d = .75)
v

array([[0.02657146],
       [0.02813005],
       [0.04752206],
       [0.02515321],
       [0.02999063],
       [0.04123711],
       [0.02657146],
       [0.04297398],
       [0.0524836 ],
       [0.07937208],
       [0.04123711],
       [0.02519154],
       [0.01030928],
       [0.02999063],
       [0.04123711],
       [0.04123711],
       [0.04003548],
       [0.02519154],
       [0.03240059],
       [0.03240059],
       [0.02515321],
       [0.05891016],
       [0.0524836 ],
       [0.07916764],
       [0.06504874]])

In [12]:
v, A = pagerank(G5,d = .6)
v

array([[0.02877   ],
       [0.03054754],
       [0.04718032],
       [0.02737374],
       [0.03152585],
       [0.04098361],
       [0.02877   ],
       [0.04152784],
       [0.05044136],
       [0.07310226],
       [0.04098361],
       [0.02735878],
       [0.01639344],
       [0.03152585],
       [0.04098361],
       [0.04098361],
       [0.03833908],
       [0.02735878],
       [0.03329918],
       [0.03329918],
       [0.02737374],
       [0.05635246],
       [0.05044136],
       [0.07320201],
       [0.06188278]])

In [13]:
v, A = pagerank(G5,d = .5)
v

array([[0.03028864],
       [0.03206084],
       [0.04661072],
       [0.02903587],
       [0.03265306],
       [0.04081633],
       [0.03028864],
       [0.04068855],
       [0.04897959],
       [0.06868832],
       [0.04081633],
       [0.0289942 ],
       [0.02040816],
       [0.03265306],
       [0.04081633],
       [0.04081633],
       [0.03762191],
       [0.0289942 ],
       [0.03401361],
       [0.03401361],
       [0.02903587],
       [0.05442177],
       [0.04897959],
       [0.06902165],
       [0.05928284]])

- Tree graph with e set to high preference for the first page.

In [14]:
e = np.ones((13, 1))
e[0] = 100
e = e / norm(e, ord = 1)

G6 = snap.GenTree(snap.PNGraph, 3, 2)
v, A = pagerank(G6, e = e, d = 0.5)
v

array([[0.86278586],
       [0.02079002],
       [0.02079002],
       [0.02079002],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601],
       [0.00831601]])

- the same graph but this time the preference is set to some other page

In [15]:
e = np.ones((13, 1))
e[12] = 100
e = e / norm(e, ord = 1)

v, A = pagerank(G6, e = e, d = 0.5)
v

array([[0.15167095],
       [0.01285347],
       [0.01285347],
       [0.26735218],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.00514139],
       [0.51413882]])

This shows that by modyfing e we can change (customize) distribution.